In [4]:
import chess
import chess.svg
import json
import os
import glob

In [ ]:
from IPython.display import display, clear_output
import ipywidgets as widgets

# Load all games from raw data
raw_dir = "../data/raw"
game_files = sorted(glob.glob(os.path.join(raw_dir, "game_*.jsonl")))
print(f"Found {len(game_files)} game(s)\n")

# Let user pick a game
games = {}
for gf in game_files:
    name = os.path.basename(gf)
    with open(gf) as f:
        games[name] = [json.loads(line) for line in f]

game_selector = widgets.Dropdown(
    options=list(games.keys()),
    description="Game:",
)
pos_slider = widgets.IntSlider(min=0, max=0, step=1, description="Move:")
board_output = widgets.Output()
info_output = widgets.Output()

def show_position(change=None):
    name = game_selector.value
    records = games[name]
    pos_slider.max = len(records) - 1
    idx = pos_slider.value
    rec = records[idx]

    board = chess.Board(rec["fen"])
    result_map = {1: "White wins", -1: "Black wins", 0: "Draw"}

    board_output.clear_output(wait=True)
    info_output.clear_output(wait=True)

    with board_output:
        display(chess.svg.board(board, size=400))

    with info_output:
        print(f"Position {idx + 1}/{len(records)}")
        print(f"Turn: {rec['current_player']}")
        print(f"MCTS value: {rec['mcts_value']}")
        print(f"Game result: {result_map.get(rec['game_result'], rec['game_result'])}")
        print(f"FEN: {rec['fen']}")
        # Show top 5 policy moves
        policy = rec.get("policy", {})
        if policy:
            top = sorted(policy.items(), key=lambda x: x[1], reverse=True)[:5]
            print("\nTop moves (visit %):")
            for move_str, prob in top:
                print(f"  {move_str}: {prob:.1%}")

game_selector.observe(show_position, names="value")
pos_slider.observe(show_position, names="value")

display(widgets.VBox([
    widgets.HBox([game_selector, pos_slider]),
    widgets.HBox([board_output, info_output])
]))
show_position()

Found 500 game(s)

